<img src="Databuds Banner.png">

<font size = "3">
   
 <br>**Created by**: Madeleine Sim 
 <br>**Contact**: madzsyl@gmail.com
 <br>**Github**: https://github.com/madzsyl/
</font>

# Who am I calling?

In this hands-on session, we will be using a public dataset - Bank Marketing data to run our analysis.
Read the instructions and fill in the answers in the accompanying form. Get all answers right and you stand a chance to win a prize (and honour and glory) in the next DataBuds!

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# magic word for producing visualizations in notebook
%matplotlib inline

#Data Source: https://archive.ics.uci.edu/ml/datasets/bank+marketing

import random
random.seed(123)

### Step 0: Load the data

In [ ]:
bankdata = pd.read_csv("bankmarketing.csv")

Note: If you are running this on Google colab, uncomment the lines below to load the dataset
Load the Databuds folder in your Google drive.
Run the following:

If the path (of bank data) does not work, you can call this code:

<font color ='orangered'>
*import os</br>
os.listdir()*</font>

to locate your files

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/')
#bankdata = pd.read_csv("drive/MyDrive/DataBuds/bankmarketing.csv")

## Data Set Information:

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed.

1) bank-additional-full.csv with all examples (41188) and 20 inputs, ordered by date (from May 2008 to November 2010), very close to the data analyzed in [Moro et al., 2014]<br>

<br>
The classification goal is to predict if the client will subscribe (yes/no) a term deposit (variable y).<br>


###  Attribute Information:
### Input variables:
#### Bank client data:
1 - age (numeric)<br>
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')<br>
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)<br>
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')<br>
5 - default: has credit in default? (categorical: 'no','yes','unknown')<br>
6 - housing: has housing loan? (categorical: 'no','yes','unknown')<br>
7 - loan: has personal loan? (categorical: 'no','yes','unknown')<br>

#### Related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')<br>
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')<br>
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')<br>
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.<br>

#### Other attributes
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)<br>
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)<br>
14 - previous: number of contacts performed before this campaign and for this client (numeric)<br>
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')<br>

#### Social and economic context attributes:
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)<br>
17 - cons.price.idx: consumer price index - monthly indicator (numeric)<br>
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)<br>
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)<br>
20 - nr.employed: number of employees - quarterly indicator (numeric)<br>

### Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')<br>

### Objective of our Clustering analysis today

Before we want to identify potential customers who are likely to subscribe to a term deposit, let's start by seeing if we can come up with some customer profiles to work with. Clustering techniques are suitable for us to create customer profiles based on their data!

In [ ]:
# Check the structure of the data after it's loaded (e.g. print the number of
# rows and columns, print the first few rows).

print("Shape of data:",bankdata.shape)
bankdata.head()

# <font color='royalblue'>Quiz 1:</font>

1. Which job forms 25% of the rows
2. How many clients have not been contacted for marketing purposes
3. True or False - Higher proportion customers with housing loan (housing = "yes") has subscribed to a term deposit (y = "yes")

Answers:</br>
(1) </br>
(2)</br>
(3)</br>

In [ ]:
bankdata.job.value_counts(normalize = True)
#bankdata.loan.value_counts() 

In [ ]:
temp = pd.pivot_table(data = bankdata, index = "housing", columns = "y", values = "loan", aggfunc='count')#.reset_index()
temp["total"] = temp["no"]+temp["yes"]
temp["no"] = round(temp["no"]/temp["total"],3)
temp["yes"] = round(temp["yes"]/temp["total"],3)
temp

In [ ]:
temp = pd.pivot_table(data = bankdata, index = "loan", columns = "y", values = "housing", aggfunc='count').reset_index()
temp["total"] = temp["no"]+temp["yes"]
temp["no"] = round(temp["no"]/temp["total"],3)
temp["yes"] = round(temp["yes"]/temp["total"],3)
temp

# Step 1: Preprocessing
### Question: 
How many columns are numeric and how many are categorical?

Answers:

In [ ]:
# 10 numeric, 11 Cat
bankdata.info()

# Note, while we wish to separate out the categorical and the numerical columns, lets do some basic conversion of columns that are simpler

### 1.1 
Drop industry level variables (we want to find out more about the customer. While industry level variables are also useful, we remove them for this analysis)

In [ ]:
bankdata.drop(columns = ["emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"], inplace = True)
bankdata.columns

Drop Month and Day variable. We could keep them if we think that customers respond differently on different periods. However, this dataset spans 3 years - while we have the Month and Day data, we do not have data on the Year. Hence, let's drop the variables

In [ ]:
bankdata.drop(columns = ["month", "day_of_week"], inplace = True)
bankdata.columns

### 1.2 Hot encode the following Variables:

In [ ]:
# job - Top 3 categories + Others
def encode_var(row, col, ori_name, cat_name, others = "Others"):
    t = others
    for i in range(len(ori_name)):
        if(row[col]==ori_name[i]):
            t = cat_name[i]
    return t

bankdata["job_new"] = bankdata.apply(lambda x: encode_var(x, "job", ["admin.", "blue-collar", "services", "management"], ["Admin", "BC", "Svcs", "Mgmt"] ), axis = 1)
bankdata[["job","job_new"]].value_counts()

bankdata["marital_new"]=bankdata.apply(lambda x: encode_var(x, "marital", ["married", "single"], ["married", "single"] ), axis = 1)
bankdata[["marital","marital_new"]].value_counts()

bankdata["education_new"]=bankdata.apply(lambda x: encode_var(x, "education", ["university.degree", "professional.course", "high.school"], ["professional", "professional", "highschool"] ), axis = 1)
bankdata[["education","education_new"]].value_counts()

bankdata["contact_tel"]=bankdata.apply(lambda x: encode_var(x, "contact", ["telephone"], [1] , others = 0), axis = 1)
bankdata[["education","education_new"]].value_counts()

bankdata["poutcome"] = bankdata.apply(lambda x: encode_var(x, "poutcome", ['failure','nonexistent','success'],[0,0,1], 0), axis = 1)

### Exercise:
Using the above example, map the variables "default", "housing", "loan" and "y" to 1 and 0

In [ ]:
#Answer
cat_yes = ["yes"]
cat_1 = [1]

bankdata["default"] = <--- Your answer--->
bankdata["housing"] = <--- Your answer--->
bankdata["loan"] = <--- Your answer--->
bankdata["y"] = <--- Your answer--->

In [ ]:
# Drop old vars
col_drop = ["job", "marital", "education", "contact"]
bankdata.drop(columns = col_drop, inplace = True)
#bankdata.info()


In [ ]:
bankdata.describe()

In [ ]:
# Hot encode categorical variables
bankdata = pd.get_dummies(bankdata)
bankdata.head()

### Step 2: Understanding Data relationships

In [ ]:
sns.set(rc={'figure.figsize':(15,12)})
sns.heatmap(bankdata.corr(),cmap = "PRGn", annot=True, fmt = "0.1f")
#bankdata.corr()

Since duration and y are reliant on each other, we remove duration now so that we do not work with data leakage if we choose to run a predictive model later

In [ ]:
col_drop = ["duration"]
bankdata.drop(columns = col_drop, inplace = True)

### 2.1 Perform Scaling

### Question for thought: Is there a need to scale our data?
Yes and No. This depends on what you want to do later. Since we wish to run a clustering, we may likely want to perform dimensionality reduction in order to optimise the runtime (very important for large sets of data).

However, dimensionality reduction will fail if the data is not scaled, as we need to pick the number of Principal Components based on the variance explained by each PC (very vulnerable to scale of data)/

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
bankdata_fit = scaler.fit_transform(bankdata)

### 2.2 Dimension Reduction Through PCA

Dimension reduction is important in cluster analysis and creates a smaller data in volume and has the same analytical results as the original representation. A clustering process needs data reduction to obtain an efficient processing time while clustering and mitigate curse of dimensionality.

Since we have many variables to work with, we can look at dimensionality reduction techniques, through PCA. 
- Use sklearn's [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) class to apply principal component analysis on the data, thus finding the vectors of maximal variance in the data. To start, you should not set any parameters (so all components are computed) or set a number of components that is at least half the number of features (so there's enough features to see the general trend in variability).
- Check out the ratio of variance explained by each principal component as well as the cumulative variance explained. Try plotting the cumulative or sequential values using matplotlib's [`plot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html) function. Based on what you find, select a value for the number of transformed features you'll retain for the clustering part of the project.
- Once you've made a choice for the number of components to keep, make sure you re-fit a PCA instance to perform the decided-on transformation.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
prin_com = pca.fit_transform(bankdata_fit)
pd.DataFrame(pca.explained_variance_ratio_).cumsum().head(10)
#(pd.DataFrame(pca.explained_variance_ratio_).cumsum().plot())

# <font color='royalblue'>Quiz 2:</font>
### If I wish to retain 65% of the data variance, how many Principal Components should I retain? 
(Hint : remember to count PC_0)

In [ ]:
## Fill in the value of N for which variance retained is at least 65%
N= <--- Your answer---> 

pca = PCA(N)
prin_com = pca.fit_transform(bankdata_fit)
#pd.DataFrame(pca.explained_variance_ratio_).cumsum()

# <font color='royalblue'>Quiz 3:</font>
### What can I say about PC 7?
a) It has more Admin professions<br>
b) It has more Management professions<br>
c) Most of them have a housing loan<br>

Answer: <--- Your answer---> 

Note: The larger the weight of the value, the more significant the variable (sign does not matter)

In [ ]:
sns.heatmap(pd.DataFrame(pca.components_, columns = bankdata.columns), annot = True,fmt = "0.1f", cmap = 'YlGnBu')

## Step 3: Clustering

### Step 3.1: Apply Clustering to the full dataset

You've assessed and cleaned the  data, then scaled and transformed them. Now, it's time to see how the data clusters in the principal components space. In this substep, you will apply k-means clustering to the dataset and use the average within-cluster distances from each point to their assigned cluster's centroid to decide on a number of clusters to keep.

- Use sklearn's [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) class to perform k-means clustering on the PCA-transformed data.
- Compute the score of the model **Hint**: The KMeans object's `.score()` method might be useful here.
- Perform the above two steps for a number of different cluster counts. You can then see how the average distance decreases with an increasing number of clusters. However, each additional cluster provides a smaller net benefit. Use this fact to select a final number of clusters in which to group the data. <br>
**Warning**: depending size of the dataset, it can take a long time for the algorithm to resolve. The more clusters to fit, the longer the algorithm will take. You should test for cluster counts through at least 10 clusters to get the full picture. Ultimately, we need to know our business needs to know how many clusters we are aiming for, and having too many clusters might hinder our understanding
- Once you've selected a final number of clusters to use, re-fit a KMeans instance to perform the clustering operation.

In [ ]:
from sklearn.cluster import KMeans
# Over a number of different cluster counts...
scores = []
centers = list(range(1,10,1))

    # run k-means clustering on the data and...
for c in centers:
    #print("Running Model for",c,"clusters")
    model_k = KMeans(n_clusters=c)
    model_k.fit(prin_com)
    # compute the average within-cluster distances.
    score = np.abs(model_k.score(prin_com))
    scores.append(score)

In [ ]:
# Investigate the change in within-cluster distance across number of clusters.
# HINT: Use matplotlib's plot function to visualize this relationship.
plt.plot(centers, scores)
plt.xticks(range(1,10))
plt.show()

In [ ]:
n_clust = 4
kmeans = KMeans(n_clusters = n_clust)
kmeans.fit(prin_com)
pred = kmeans.predict(prin_com)

## Step 4: Making sense of your clusters

### 1st way: Reversing the scaling and PC of your data to study the Centroids of each cluster

In [ ]:
centroid_0 = scaler.inverse_transform(pca.inverse_transform(kmeans.cluster_centers_[0]))
over_c = pd.Series(data = centroid_0, index = bankdata.columns)
over_c

### 2nd Way: Call out the mean of a groupby("Pred") object 
(a lot easier! I would prefer this because it's straightforward)<br>
Recap: We assigned the cluster numbers to the variable "Pred"

In [ ]:
bankdata["pred"] = pred
bankdata.groupby("pred").mean()

# <font color='royalblue'>Quiz 4:</font>
### Which of these statements are likely true?
a) cluster 0 has the oldest customers</br>
b) cluster 1 comprises the most customers who were recently contacted</br>
c) cluster 1 has had the most defaults previously</br>
d) cluster 1 has a higher proportion of married customers</br>
Answer:

# The end!!

This sums up the gist of how you would run an unsupervised clustering from start to end! While the steps taken can be pretty standard, it is improtant to understand your business context and data points before making decisions on how to manipulate the data!

Feel free to substitute the scaling, dimensionality reduction, and clustering model with different methods to see how your data changes!

## What... next?
Notice that in the dataset, a variable "y" exists. This variable tells us if a client subscribes to a term deposit. We could possibly build a **classification model**, if we wish to predict which customers are more likely to subscribe to a term deposit (and hence, spend our time calling them


In [ ]:
## An Implementation of DBScan

from sklearn.cluster import DBSCAN  

db_cluster = DBSCAN(eps = 0.5, min_samples = 3).fit(prin_com)
pred = db_cluster.labels_

In [ ]:
## An Implementation of Gaussian Mixed Models
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=5)
gmm_pred = gmm.fit_predict(prin_com)

## If you want the probabilities of each data point belonging to the cluster, you can do this:
# gmm_predict_proba = gmm.fit_predict_proba(prin_com)